# Content Based Recommendation System 

In [5]:
# -*- coding: utf-8 -*-
"""
Created on Sat Dec 21 20:40:57 2019

@author: Abdulrahman Alothman
"""

import numpy as np # linear algebra
import pandas as pd # data processing

rating = pd.read_csv('rating.csv',encoding = "latin-1")
anime = pd.read_csv('anime.csv')
anime.head(8)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679


In [7]:
#control Dataset if it has nan values 
anime = anime.astype({"genre":'category',"episodes":'category','type':'category' })
print(anime.isnull().sum().to_frame('nulls'))

          nulls
anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0


# Complete the missing data

In [9]:
#I will use the 'Interpolation' method and most frequncy value to complete the missing data below

anime.rating=anime.rating.fillna(anime.rating.mean())
anime["genre"]=(anime["genre"].cat.codes.replace(-1, np.nan).interpolate().astype(int).astype('category').cat.rename_categories(anime["genre"].cat.categories))
anime["type"]=(anime["type"].cat.codes.replace(-1, np.nan).interpolate().astype(int).astype('category').cat.rename_categories(anime["type"].cat.categories))
print(anime.isnull().sum().to_frame('nulls'))

          nulls
anime_id      0
name          0
genre         0
type          0
episodes      0
rating        0
members       0


In [3]:
veriler = df_rating.drop_duplicates(subset=['anime_id'])
veriler.head(10)

,user_id,anime_id,rating,name,genre,type,episodes,members
1,3,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
25926,5,24,1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,178553
33081,102,79,7,Shuffle!,"Comedy, Drama, Ecchi, Fantasy, Harem, Magic, R...",TV,24,158772
40193,17,226,7,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,623511
67697,5,241,1,Girls Bravo: First Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,11,84395
71877,8,355,9,Shakugan no Shana,"Action, Drama, Fantasy, Romance, School, Super...",TV,24,297058
83827,21,356,8,Fate/stay night,"Action, Fantasy, Magic, Romance, Supernatural",TV,24,374880
98742,5,442,4,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Drama, Historical, Shounen,...",Movie,1,120571
106588,5,487,1,Girls Bravo: Second Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,13,57110
109798,5,846,1,School Rumble Ni Gakki,"Comedy, Romance, School, Shounen",TV,26,93095


# Genre Based Recommender
We will compute pairwise similarity scores for all movies based on their genre and recommend movies based on that 
similarity score. Tfidfvectorizer you compute the word counts, idf and tf-idf values all at once.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(veriler['genre'])
print(tfidf_matrix.shape)


(9926, 46)


In [11]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#Construct a reverse map of indices and movie titles
indices = pd.Series(veriler.index, index=veriler['name']).drop_duplicates()

In [26]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(name, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[name]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
 
    # Return the top 10 most similar movies
    
    return veriler['name'].iloc[movie_indices]

#naruto'ye benzeyen Filmler
result=get_recommendations('Naruto')
result=pd.DataFrame(result)
onerilen_filmler=veriler[veriler.name.isin(result['name'])]
onerilen_filmler[['name','genre']]

                                      name  \
109798              School Rumble Ni Gakki   
2096931    School Rumble Ichi Gakki Hoshuu   
2329776            School Rumble San Gakki   
3102247  Yamada-kun to 7-nin no Majo (OVA)   
3816307                 Denpa Kyoushi (TV)   
5855903           Hori-san to Miyamura-kun   
6227831                   Hatsukoi Limited   
7060450                   Megane na Kanojo   
7778525                        The Samurai   
7811608                      Shibai Taroka   

                                            genre  
109798           Comedy, Romance, School, Shounen  
2096931          Comedy, Romance, School, Shounen  
2329776          Comedy, Romance, School, Shounen  
3102247          Comedy, Romance, School, Shounen  
3816307          Comedy, Romance, School, Shounen  
5855903          Comedy, Romance, School, Shounen  
6227831          Comedy, Romance, School, Shounen  
7060450          Comedy, Romance, School, Shounen  
7778525          Comedy, 